The goal of this notebook is to give a simple demo on how one could turn their natural language instructions into SQL query and then query a local database. This tutorial can be easily extended to databases that utilize client-server architecture.

This tutorial uses ChatGPT API to convert natural language to queries. For users not comfortable sending their personal queries to OpenAI, we have also demonstrated the same use case by using local Gemma model weights.

Additionally, we have added a bonus section whereby the LLM would be able to understand the existing schema of database and would avoid hallucinating any information regarding any tables.

## This notebook is divided into following sections - <br>

1. **Database Creation** - Create a SQLite3 DB and insert some data in it. Test some basic queries.

2. **Setup ChatGPT API connection** - Connect to OpenAI's GPT-4 model to translate natural language into SQL queries.

3. **Write prompts and query DB** - Write prompts and enhance them to get good results from LLM. Execute SQL queries on local DB.

4. **Prevent hallucination** - Describe the DB schema in prompt so that the LLM does not hallucinates.

5. **Download LLM locally** - Download Google's Gemma model weights to local machine and use the LLM.

# Database Creation

SQLite3 is the library we are going to use to create and query a DB.It is a built-in library so you may not need to install it.

Let us first setup a simple database and insert some data.

In [1]:
import sqlite3

In [2]:
# Connect to a SQLite database. This will create a database with the given name if it does not exists.
conn = sqlite3.connect('student.db')

In [3]:
ls

sample_data/  student.db


In [4]:
# Create a cursor object
cursor = conn.cursor()

In [5]:
# Create a table
cursor.execute('''
CREATE TABLE IF NOT EXISTS student (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER NOT NULL,
    gender TEXT NOT NULL,
    grade TEXT NOT NULL
)
''')

In [6]:
# Insert some data rows in the table
cursor.execute("INSERT INTO student (id, name, age, gender, grade) VALUES (101,'Alice', 17, 'female', 'senior')")
cursor.execute("INSERT INTO student (id, name, age, gender, grade) VALUES (102,'Bob', 14, 'male','freshman')")
cursor.execute("INSERT INTO student (id, name, age, gender, grade) VALUES (103,'Charlie', 15, 'male', 'sophomore')")
cursor.execute("INSERT INTO student (id, name, age, gender, grade) VALUES (104, 'Lily', 7, 'female', 'elementary')")

#Commit the changes
conn.commit()

In [7]:
# Query the database
cursor.execute("SELECT name, age FROM student WHERE age > 14")

# Fetch all the results
results = cursor.fetchall()

# Display the results
for row in results:
  print(row)

('Alice', 17)
('Charlie', 15)


In [8]:
# Close the cursor and connection
cursor.close()
conn.close()

Lets write a function which can be used to get query results

In [9]:
def execute_sql_query(sql_query):
    # Connect to SQLite database
    conn = sqlite3.connect('student.db')
    cursor = conn.cursor()

    try:
        # Execute the SQL query
        cursor.execute(sql_query)
        columns = [column[0] for column in cursor.description]
        results = cursor.fetchall()
        #print("Query executed successfully. Here are the results:")
        #for row in results:
        #    print(row)
        rows = [dict(zip(columns,row)) for row in results]
        return rows
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the connection
        if cursor:
          cursor.close()
        if conn:
          conn.close()

In [10]:
execute_sql_query("SELECT name, age FROM student WHERE age > 14")

[{'name': 'Alice', 'age': 17}, {'name': 'Charlie', 'age': 15}]

Great! So far we have our database set up and an easy way to query it. Let's connect to ChatGPT.

# Setup ChatGPT API connection

Thus far, we have successfully created a simple DB with four rows in it. Now we will try to move our focus towards generating a SQL query using LLM model.

We will connect to OpenAI's GPT-4 model and ask a simple question.

In [ ]:
!pip install openai

In [12]:

from openai import OpenAI
from getpass import getpass

#Input the API key
OPENAI_API_KEY = getpass("OpenAI API Key: ")

#Define the model
model = "gpt-4" # You can also try "gpt-3.5-turbo"

#Connect to client
client = OpenAI(api_key=OPENAI_API_KEY)

#Generate response for your query
response = client.chat.completions.create(
  model=model,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is a LLM?"}
  ],
  max_tokens=500,
  temperature=0.7 #0 means deterministic. 1 is the default value.
)

OpenAI API Key: ··········


In [13]:
response.choices[0].message.content

"An LLM, or Master of Laws, is an advanced degree in law that is often pursued by students who want to specialize in a specific area of law. It usually requires a bachelor's degree in law (LLB) as a prerequisite and is typically a one-year full-time study program. Some of the specializations could include international law, tax law, environmental law, human rights law, and more. It's particularly popular among international students who want to familiarize themselves with the legal system of a particular country."

# Write prompts and query DB

We will write a function that converts natural text to query.

In [14]:
from openai import OpenAI
from getpass import getpass

OPENAI_API_KEY = getpass("OpenAI API Key: ")

client = OpenAI(api_key=OPENAI_API_KEY)

OpenAI API Key: ··········


In [15]:
def convert_text_to_query(input_text,user_prompt,system_prompt="You are a helpful assistant.",model="gpt-4"):

  response = client.chat.completions.create(
  model=model,
  messages=[
    {"role": "system", "content": system_prompt },
    {"role": "user", "content": user_prompt.format(natural_language_query=input_text)}
  ],
  max_tokens=500,
  temperature=0.7 #0 means deterministic. 1 is the default value.
  )

  return response.choices[0].message.content


In [16]:
system_prompt = """
You are a helpful assistant that converts natural language into SQL queries.
"""

user_prompt = """
Convert the following text into an SQL query:{natural_language_query}.
"""

input_text = "How many students are of above 7 years of age?"

In [17]:
query = convert_text_to_query(input_text,user_prompt=user_prompt,system_prompt=system_prompt)

In [18]:
query

'SELECT COUNT(*) \nFROM students \nWHERE age > 7;'

In [19]:
execute_sql_query(query)

An error occurred: no such table: students


You see! The LLM has no idea about the tables we have in our database. Let's enhance or query to accomodate this and also add a few examples.

In [20]:
system_prompt = """
You are a helpful assistant that converts natural language into SQL queries.
"""

user_prompt ="""
You are a highly skilled SQL translator. Your task is to convert natural language descriptions of data queries into correct and optimized SQL statements.

Here are a few examples:

1. **Input**: "Select the names and salaries of all employees in the 'Finance' department."
   **Output**:

   SELECT name, salary
   FROM employees
   WHERE department = 'Finance';

2. **Input**: "Find the total number of orders placed by customers in 2023."
   **Output**:

   SELECT COUNT(*)
   FROM orders
   WHERE YEAR(order_date) = 2023;

3. **Input**: "Retrieve the email addresses of customers who have not placed any orders."
   **Output**:

   SELECT email
   FROM customers
   WHERE customer_id NOT IN (SELECT customer_id FROM orders);

4. **Input**: "List the product names and their average rating for products with more than 100 reviews."
   **Output**:

   SELECT product_name, AVG(rating) AS average_rating
   FROM reviews
   GROUP BY product_name
   HAVING COUNT(*) > 100;

5. **Input**: "Get the total sales and the number of orders for each product sold in January 2024."
   **Output**:

   SELECT product_id, SUM(sales_amount) AS total_sales, COUNT(order_id) AS order_count
   FROM sales
   WHERE order_date BETWEEN '2024-01-01' AND '2024-01-31'
   GROUP BY product_id;

Now, translate the following natural language query into an SQL query:
**Input**: {natural_language_query}
**Output**:

"""
input_text = """
How many students are of above 7 years of age? The details can be found in table 'student'
"""

In [21]:
query = convert_text_to_query(input_text,user_prompt=user_prompt,system_prompt=system_prompt)

In [22]:
execute_sql_query(query)

[{'COUNT(*)': 3}]

# Prevent hallucination

Did you notice how in the previous section the LLM hallucinated the table name 'students' ? It will be a bit hectic for us if we have to mention all the table and column names everytime we want to ask a question. A good idea would be to automate the insertion of the metadata information on tables and the corresponding columns in the prompt itself.

For this, we would create two tables -
1. **table_metadata** - Stores the table description.
2. **column_metadata** - Stores the column description for each table.

Let's create them and insert information in them.

In [23]:
conn = sqlite3.connect('student.db')

In [24]:
# Create a cursor object
cursor = conn.cursor()

First create a table that stores the meta data about the table.

In [25]:
# Create a table
cursor.execute('''
CREATE TABLE IF NOT EXISTS table_metadata (
    table_name TEXT PRIMARY KEY,
    description TEXT
)
''')

In [26]:
cursor.execute("INSERT INTO table_metadata (table_name, description) VALUES ('student','Details of students in school.')")
conn.commit()

In [27]:
execute_sql_query("SELECT * FROM table_metadata")

[{'table_name': 'student', 'description': 'Details of students in school.'}]

Create a table that stores metadata about the columns in each table.

In [28]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS column_metadata (
  table_name TEXT,
  column_name TEXT,
  data_type TEXT,
  description TEXT,
  PRIMARY KEY (table_name, column_name),
  FOREIGN KEY (table_name) references table_metadata (table_name)
)
''')

In [29]:
cursor.execute("INSERT INTO column_metadata (table_name, column_name, data_type,description) VALUES ('student','id','INTEGER','Identification number of the student.')")
cursor.execute("INSERT INTO column_metadata (table_name, column_name, data_type,description) VALUES ('student','name','TEXT','Name of the student.')")
cursor.execute("INSERT INTO column_metadata (table_name, column_name, data_type,description) VALUES ('student','age','INTEGER','Age of the student.')")
cursor.execute("INSERT INTO column_metadata (table_name, column_name, data_type,description) VALUES ('student','gender','TEXT','Gender of the student.')")
cursor.execute("INSERT INTO column_metadata (table_name, column_name, data_type,description) VALUES ('student','grade','TEXT','Grade in which the student studies.')")
conn.commit()

In [30]:
execute_sql_query("SELECT * FROM column_metadata")

[{'table_name': 'student',
  'column_name': 'id',
  'data_type': 'INTEGER',
  'description': 'Identification number of the student.'},
 {'table_name': 'student',
  'column_name': 'name',
  'data_type': 'TEXT',
  'description': 'Name of the student.'},
 {'table_name': 'student',
  'column_name': 'age',
  'data_type': 'INTEGER',
  'description': 'Age of the student.'},
 {'table_name': 'student',
  'column_name': 'gender',
  'data_type': 'TEXT',
  'description': 'Gender of the student.'},
 {'table_name': 'student',
  'column_name': 'grade',
  'data_type': 'TEXT',
  'description': 'Grade in which the student studies.'}]

In [31]:
cursor.close()
conn.close()

Great! So how do we insert all this schema information in the prompt? Let's write a function that does that.

In [38]:
def get_table_info(table_name):

    query_table = "SELECT * FROM table_metadata where table_name = '{table_name}';".format(table_name=table_name)
    table_info = execute_sql_query(query_table)[0]
    table_name = table_info["table_name"]
    description = table_info["description"]

    result = """\n
    The table name is '{table_name}'
    and the description is as follows:
    '{description}'
    \n
    The column information for table {table_name} is as follows :\n
    """.format(table_name=table_name,description=description)


    result += "{:<15} {:<15} {:<15}\n".format("Column Name", "Data Type", "Description")

    query_columns = "SELECT * FROM column_metadata where table_name = '{table_name}';".format(table_name=table_name)
    columns_info = execute_sql_query(query_columns)

    for column in columns_info:
      column_name = column["column_name"]
      data_type = column["data_type"]
      description = column["description"]
      result += f"{column_name:<15} {data_type:<15} {description:<15}\n"


    return result


The above function will get you the table information given a table's name. But we can have multiple tables. So let's get all the table information using another function

In [39]:
def get_schema_information():

  result = """
  The name of database is student.db and following is the information on its tables.\n
  """

  query_schema = "SELECT * FROM table_metadata;"

  tables = execute_sql_query(query_schema)

  for table in tables:
    result += get_table_info(table["table_name"])

  return result



In [40]:
result = get_schema_information()
print(result)


  The name of database is student.db and following is the information on its tables.

  

    The table name is 'student'
    and the description is as follows:
    'Details of students in school.'
    

    The column information for table student is as follows :

    Column Name     Data Type       Description    
age             INTEGER         Age of the student.
gender          TEXT            Gender of the student.
grade           TEXT            Grade in which the student studies.
id              INTEGER         Identification number of the student.
name            TEXT            Name of the student.



Now, we will enhance our prompt with this information. For that, lets write another function that converts text to query but using schema information.

In [41]:
def convert_text_to_query_with_schema(input_text,schema_info,user_prompt,system_prompt="You are a helpful assistant.",model="gpt-4"):

  response = client.chat.completions.create(
  model=model,
  messages=[
    {"role": "system", "content": system_prompt },
    {"role": "user", "content": user_prompt.format(natural_language_query=input_text,schema_info=schema_info)}
  ],
  max_tokens=500,
  temperature=0.7 #0 means deterministic. 1 is the default value.
  )

  return response.choices[0].message.content

Let's adjust our prompt a bit.

In [42]:
system_prompt = """
You are a helpful assistant that converts natural language into SQL queries.
"""

user_prompt ="""
You are a highly skilled SQL translator. Your task is to convert natural language descriptions of data queries into correct and optimized SQL statements.

Here are a few examples:

1. **Input**: "Select the names and salaries of all employees in the 'Finance' department."
   **Output**:

   SELECT name, salary
   FROM employees
   WHERE department = 'Finance';

2. **Input**: "Find the total number of orders placed by customers in 2023."
   **Output**:

   SELECT COUNT(*)
   FROM orders
   WHERE YEAR(order_date) = 2023;

3. **Input**: "Retrieve the email addresses of customers who have not placed any orders."
   **Output**:

   SELECT email
   FROM customers
   WHERE customer_id NOT IN (SELECT customer_id FROM orders);

4. **Input**: "List the product names and their average rating for products with more than 100 reviews."
   **Output**:

   SELECT product_name, AVG(rating) AS average_rating
   FROM reviews
   GROUP BY product_name
   HAVING COUNT(*) > 100;

5. **Input**: "Get the total sales and the number of orders for each product sold in January 2024."
   **Output**:

   SELECT product_id, SUM(sales_amount) AS total_sales, COUNT(order_id) AS order_count
   FROM sales
   WHERE order_date BETWEEN '2024-01-01' AND '2024-01-31'
   GROUP BY product_id;

Here is the schema information for our database :
{schema_info}

Please only answer queries which makes sense for the given schema. Else just return - "No information found"

Now, translate the following natural language query into an SQL query:
**Input**: {natural_language_query}
**Output**:

"""


It is essentially the same except that we have added the following information -<br>
>'''Here is the schema information for our database :<br>
>{schema_info}

>Please only answer queries which makes sense for the given schema. Else just return - "No information found"
'''

In [43]:
input_text = """
How many students are of above 7 years of age?
"""

In [44]:
schema_info = get_schema_information()
query = convert_text_to_query_with_schema(input_text,schema_info,user_prompt,system_prompt)

In [45]:
query

'SELECT COUNT(*)\nFROM student\nWHERE age > 7;'

In [46]:
execute_sql_query(query)

[{'COUNT(*)': 3}]

Viola! It works great!

Now, lets try to ask something data that is not present in database.

In [47]:
input_text = """
How many teachers teach in the school?
"""

In [48]:
schema_info = get_schema_information()
query = convert_text_to_query_with_schema(input_text,schema_info,user_prompt,system_prompt)

In [49]:
query

'No information found'

You see! The LLM returns 'No information found'.

#  Download LLM weights locally

Many times we may not be comfortable exposing the database schema information with an alien LLM. Who knows where this information can be used and how? So what we will do is download the weights of a LLM and use it to produce results locally.

We are going to use Google' Gemma model with 7 billion parameters. The information on this can be found [here](https://huggingface.co/google/gemma-7b).

Please note that there are other good models out there. The best ones are not open sourced and hence, cannot be downloaded for example GPT-4. Some have too much many parameters to download on a single GPU.

We need to have a GPU to run this model locally. I am using Google Colab's L4 GPU.

Much of the following code is reused from Daniel Bourke's fantastic [tutorial](https://github.com/mrdbourke/simple-local-rag)

Firts let's import some libraries

In [50]:
import os
if "COLAB_GPU" in os.environ:
    !pip install -U torch # requires torch 2.1.1+ (for efficient sdpa implementation)
    !pip install accelerate # for quantization model loading
    !pip install flash-attn --no-build-isolation # for faster attention mechanism = faster LLM inference

  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde4ba609a21ebb98382377a27041edc09318801a6f24
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash-attn


Let's see how much memory have we got

In [51]:
!nvidia-smi

Sun Aug 18 04:52:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   42C    P8              16W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [52]:
#Let's verify it with code.
# Get GPU available memory
import torch
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")

Available GPU memory: 22 GB


We got around 22 GB of memory. Our model has 7 billion parameters and would need approximately 19 GB of memory

The following table shows different amounts of GPU memory requirements for different verions of the Gemma LLMs with varying levels of precision.

| Model             | Precision | Min-Memory (Bytes) | Min-Memory (MB) | Min-Memory (GB) | Recommended Memory (GB) | Hugging Face ID |
|-------------------|-----------|----------------|-------------|-------------| ----- | ----- |
| [Gemma 2B](https://huggingface.co/google/gemma-2b-it)          | 4-bit     | 2,106,749,952  | 2009.15     | 1.96        | ~5.0 | [`gemma-2b`](https://huggingface.co/google/gemma-2b) or [`gemma-2b-it`](https://huggingface.co/google/gemma-2b-it) for instruction tuned version |
| Gemma 2B          | Float16   | 5,079,453,696  | 4844.14     | 4.73        | ~8.0 | Same as above |
| [Gemma 7B](https://huggingface.co/google/gemma-7b-it)          | 4-bit     | 5,515,859,968  | 5260.33     | 5.14        | ~8.0 | [`gemma-7b`](https://huggingface.co/google/gemma-7b) or [`gemma-7b-it`](https://huggingface.co/google/gemma-7b-it) for instruction tuned version |
| Gemma 7B          | Float16   | 17,142,470,656 | 16348.33    | 15.97       | ~19 | Same as above |


We are using a hugging face model. Let's login to that. If you dont have an account already, please create one. You might need a key to authenticate yourself.

In [ ]:
!huggingface-cli login

Let's check my username

In [ ]:
!huggingface-cli whoami

In [54]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available

# Setup Flash Attention 2 for faster inference, default to "sdpa" or "scaled dot product attention" if it's not available
# Flash Attention 2 requires NVIDIA GPU compute capability of 8.0 or above, see: https://developer.nvidia.com/cuda-gpus
# Requires !pip install flash-attn, see: https://github.com/Dao-AILab/flash-attention
if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
  attn_implementation = "flash_attention_2"
else:
  attn_implementation = "sdpa"
print(f"[INFO] Using attention implementation: {attn_implementation}")

# 1. Pick a model we'd like to use (this will depend on how much GPU memory you have available)

model_id = "google/gemma-7b-it"
print(f"[INFO] Using model_id: {model_id}")

# 2. Instantiate tokenizer (tokenizer turns text into numbers ready for the model)
model_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)

# 3. Instantiate the model
llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id,
                                                 torch_dtype=torch.float16, # datatype to use, we want float16
                                                 quantization_config=None,
                                                 low_cpu_mem_usage=False, # use full memory
                                                 attn_implementation=attn_implementation) # which attention version to use

llm_model.to("cuda")



[INFO] Using attention implementation: flash_attention_2
[INFO] Using model_id: google/gemma-7b-it


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 3072, padding_idx=0)
    (layers): ModuleList(
      (0-27): 28 x GemmaDecoderLayer(
        (self_attn): GemmaFlashAttention2(
          (q_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (v_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3072, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (up_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (down_proj): Linear(in_features=24576, out_features=3072, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): Ge

Let's check our model's architecture

In [55]:
llm_model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 3072, padding_idx=0)
    (layers): ModuleList(
      (0-27): 28 x GemmaDecoderLayer(
        (self_attn): GemmaFlashAttention2(
          (q_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (v_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3072, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (up_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (down_proj): Linear(in_features=24576, out_features=3072, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): Ge

How about we get number of parameters in our model?

In [56]:
def get_model_num_params(model: torch.nn.Module):
    return sum([param.numel() for param in model.parameters()])

get_model_num_params(llm_model)

8537680896

How about we get model memory requirements?

In [57]:
def get_model_mem_size(model: torch.nn.Module):
    """
    Get how much memory a PyTorch model takes up.

    See: https://discuss.pytorch.org/t/gpu-memory-that-model-uses/56822
    """
    # Get model parameters and buffer sizes
    mem_params = sum([param.nelement() * param.element_size() for param in model.parameters()])
    mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])

    # Calculate various model sizes
    model_mem_bytes = mem_params + mem_buffers # in bytes
    model_mem_mb = model_mem_bytes / (1024**2) # in megabytes
    model_mem_gb = model_mem_bytes / (1024**3) # in gigabytes

    return {"model_mem_bytes": model_mem_bytes,
            "model_mem_mb": round(model_mem_mb, 2),
            "model_mem_gb": round(model_mem_gb, 2)}

get_model_mem_size(llm_model)

{'model_mem_bytes': 17075376128,
 'model_mem_mb': 16284.35,
 'model_mem_gb': 15.9}

Generating text with LLM

In [58]:
input_text = "What is LLM?"
print(f"Input text:\n{input_text}")

# Create prompt template for instruction-tuned model
dialogue_template = [
    {"role": "user",
     "content": input_text}
]

# Apply the chat template
prompt = model_tokenizer.apply_chat_template(conversation=dialogue_template,
                                       tokenize=False, # keep as raw text (not tokenized)
                                       add_generation_prompt=True)
print(f"\nPrompt (formatted):\n{prompt}")

Input text:
What is LLM?

Prompt (formatted):
<bos><start_of_turn>user
What is LLM?<end_of_turn>
<start_of_turn>model



In [59]:
%%time

# Tokenize the input text (turn it into numbers) and send it to GPU
input_ids = model_tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to("cuda")
print(f"Model input (tokenized):\n{input_ids}\n")

# Generate outputs passed on the tokenized input
# See generate docs: https://huggingface.co/docs/transformers/v4.38.2/en/main_classes/text_generation#transformers.GenerationConfig
outputs = llm_model.generate(input_ids=input_ids.to(llm_model.device),
                             max_new_tokens=512) # define the maximum number of new tokens to create
print(f"Model output (tokens):\n{outputs[0]}\n")

Model input (tokenized):
tensor([[     2,    106,   1645,    108,   1841,    603,    629,  18622, 235336,
            107,    108,    106,   2516,    108]], device='cuda:0')

Model output (tokens):
tensor([     2,    106,   1645,    108,   1841,    603,    629,  18622, 235336,
           107,    108,    106,   2516,    108,   1650, 235296,  12353,    604,
         14944,  18454,  14944,   5708, 235265,   1165,    603,    476,   1916,
           576,   2910,   5255,   2091,    591,   1650, 235296, 235275,    674,
           603,  14076,   6869,    577,   3508,    578,  11941,   3515, 235290,
          5023,   2793, 235265,  25599,  14816,    708,  15976,  17363,    611,
         12380,  15992,    576,   2793,   1423, 235269,    578,    984,    708,
          3326,    577,  18254,    575,    476,   5396,   3001,    576,  13333,
        235269,   3359,   2793,  20823,   2465, 235269,  17183, 235269,   3409,
         10304, 235269,    578,   2872,  39534, 235265,      1],
       device='cu

In [60]:
# Decode the output tokens to text
outputs_decoded = model_tokenizer.decode(outputs[0])
print(f"Model output (decoded):\n{outputs_decoded}\n")

Model output (decoded):
<bos><start_of_turn>user
What is LLM?<end_of_turn>
<start_of_turn>model
LLM stands for Language Large Language Model. It is a type of large language model (LLM) that is specifically designed to understand and generate human-like text. LLMs are typically trained on vast amounts of text data, and they are able to engage in a wide range of tasks, including text summarization, translation, code generation, and question answering.<eos>



Let's write a function to use this LLM and generate query. We will tweak our function 'convert_text_to_query_with_schema' a bit to suit Google's Gemma.

In [61]:
def convert_text_to_query_locally(input_text,schema_info,user_prompt):

  # Create prompt template for instruction-tuned model
  dialogue_template = [
      {"role": "user", "content": user_prompt.format(natural_language_query=input_text,schema_info=schema_info)}
  ]

  # Apply the chat template
  prompt = model_tokenizer.apply_chat_template(conversation=dialogue_template,
                                        tokenize=False, # keep as raw text (not tokenized)
                                        add_generation_prompt=True)



  # Tokenize the input text (turn it into numbers) and send it to GPU
  input_ids = model_tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to("cuda")


  # Generate outputs passed on the tokenized input

  outputs = llm_model.generate(input_ids=input_ids.to(llm_model.device),
                              max_new_tokens=512)

  llm_output = model_tokenizer.decode(outputs[0])

  llm_output = llm_output.replace(prompt,"").replace("\n"," ")



  return llm_output



Notice how the Gemma does not take "system" role and hence it is not part of our prompt.

In [62]:


user_prompt ="""
You are a highly skilled SQL translator. Your task is to convert natural language descriptions of data queries into correct and optimized SQL statements.

Here are a few examples:

1. **Input**: "Select the names and salaries of all employees in the 'Finance' department."
   **Output**:

   SELECT name, salary
   FROM employees
   WHERE department = 'Finance';

2. **Input**: "Find the total number of orders placed by customers in 2023."
   **Output**:

   SELECT COUNT(*)
   FROM orders
   WHERE YEAR(order_date) = 2023;

3. **Input**: "Retrieve the email addresses of customers who have not placed any orders."
   **Output**:

   SELECT email
   FROM customers
   WHERE customer_id NOT IN (SELECT customer_id FROM orders);

4. **Input**: "List the product names and their average rating for products with more than 100 reviews."
   **Output**:

   SELECT product_name, AVG(rating) AS average_rating
   FROM reviews
   GROUP BY product_name
   HAVING COUNT(*) > 100;

5. **Input**: "Get the total sales and the number of orders for each product sold in January 2024."
   **Output**:

   SELECT product_id, SUM(sales_amount) AS total_sales, COUNT(order_id) AS order_count
   FROM sales
   WHERE order_date BETWEEN '2024-01-01' AND '2024-01-31'
   GROUP BY product_id;

Here is the schema information for our database :
{schema_info}

Please do not return additional text besides query.

Please only answer queries which makes sense for the given schema. Else just return - "No information found"

Now, translate the following natural language query into an SQL query:
**Input**: {natural_language_query}
**Output**:

"""

In [63]:
input_text = """
How many students are of above 7 years of age?
"""

In [64]:
schema_info = get_schema_information()
query = convert_text_to_query_locally(input_text,schema_info,user_prompt)

In [65]:
query

'Sure, here is the translated SQL query:  SELECT COUNT(*) FROM student WHERE age > 7;<eos>'

Let's do little cleaning

In [66]:
import re

def clean_query(query):

  if("no information found" in query.lower()):
    return "No information found"

  query =  query.replace("<eos>","")

  pattern = r'(select.*?;)'

  match = re.search(pattern, query, re.IGNORECASE)

  # Extract and print the matched substring
  if match:
      extracted_query = match.group(1)
      return extracted_query
  else:
      print("No matching query found.")



In [68]:
query = clean_query(query)

In [69]:
execute_sql_query(query)

[{'COUNT(*)': 3}]

Let's try to trick it to hallucinate

In [70]:
input_text = """
How many teachers are there in the school?
"""
schema_info = get_schema_information()
query = convert_text_to_query_locally(input_text,schema_info,user_prompt)
query = clean_query(query)

In [71]:
query

"SELECT COUNT(*) FROM teachers WHERE school_id = (SELECT id FROM schools WHERE name = 'My School');"

In [72]:
execute_sql_query(query)

An error occurred: no such table: teachers


You see! This model can be easily be hallucinated even though we have provided it with schema information. This is the drawback of using smaller LLMs.

In the subsequent tutorial, we will try to use some simple and quick finetuning to fix this issue.